In [5]:
import torch
from dct_lee import DCT, IDCT
a = torch.tensor([1.,2.,3.,4.,5.,6.,7.,8.]).cuda()
dct = DCT().cuda()
idct = IDCT().cuda()
b = dct(a)
ab = idct(b)
print(b)
print(ab)

tensor([ 9.0000, -3.2212,  0.0000, -0.3367,  0.0000, -0.1005,  0.0000, -0.0254],
       device='cuda:0')
tensor([ 2.0000,  4.0000,  6.0000,  8.0000, 10.0000, 12.0000, 14.0000, 16.0000],
       device='cuda:0')


In [7]:
import numpy as np
from scipy.fft import dct as sci_dct
from scipy.fft import idct as sci_idct
aa = np.array([1.,2.,3.,4.,5.,6.,7.,8.])
bb = sci_dct(aa, norm='ortho')
aabb = sci_idct(bb, norm='ortho')
print(bb)
print(aabb)
# match with matlab

[12.72792206 -6.44232302  0.         -0.6734548   0.         -0.2009029
  0.         -0.05070232]
[1. 2. 3. 4. 5. 6. 7. 8.]
